# Multilingual & Cross‑Lingual Prompting Lab
We use **mT5‑base** to translate and answer questions in three languages.

## 1. Install & load

In [ ]:
!pip -q install --upgrade transformers sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name='google/mt5-base'
tok=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)


## 2. Helper to generate

In [ ]:
def generate(prompt, lang_tag='<extra_id_0>'):
    # Incorporate lang_tag as a prefix to guide multilingual generation
    if lang_tag and lang_tag != '<extra_id_0>':
        full_prompt = f'{lang_tag} {prompt}'
    else:
        full_prompt = prompt
    
    input_ids=tok.encode(full_prompt, return_tensors='pt')
    out=model.generate(input_ids, max_new_tokens=64)
    return tok.decode(out[0], skip_special_tokens=True)


### Demo: Using lang_tag parameter
The `lang_tag` parameter can be used to prefix prompts with language-specific instructions.

In [ ]:
# Example with lang_tag parameter
text = 'The weather is beautiful today.'

# Using lang_tag for different translation tasks
print('Spanish:', generate(text, lang_tag='translate English to Spanish:'))
print('French:', generate(text, lang_tag='translate English to French:'))
print('German:', generate(text, lang_tag='translate English to German:'))

### Task: Summarise a news headline in three languages

In [ ]:
headline='Scientists discover a new exoplanet resembling Earth.'

# Original approach: concatenating prefix with headline
print('=== Original approach ===')
for lang,tag in [('English',''),('Spanish','translate English to Spanish: '),('Chinese','translate English to Chinese: ')]:
    print(lang, generate(tag+headline))

print('\n=== Using lang_tag parameter ===')
# New approach: using lang_tag parameter for cleaner separation
for lang,tag in [('English',''),('Spanish','translate English to Spanish:'),('Chinese','translate English to Chinese:')]:
    print(lang, generate(headline, lang_tag=tag))

## 3. Cross‑lingual Q&A
Ask in English, answer in German.

In [ ]:
q='When was the Great Wall of China built?'

# Original approach
print('=== Original approach ===')
prompt=f'translate English to German: Beantworte die Frage: {q}'
print(generate(prompt))

# Using lang_tag parameter
print('\n=== Using lang_tag parameter ===')
print(generate(f'Beantworte die Frage: {q}', lang_tag='translate English to German:'))

### Exercise
1. Craft a *single* prompt template that detects input language and responds in that language.
2. Evaluate fluency with BLEU or COMET (advanced).
3. **New**: Experiment with the `lang_tag` parameter to create more modular multilingual prompts.
4. **New**: Try combining multiple language instructions in the `lang_tag` parameter.

### Advanced lang_tag Examples
Explore more sophisticated uses of the `lang_tag` parameter.

In [ ]:
# Example: Complex multilingual tasks with lang_tag
story = 'Once upon a time, there was a brave knight who saved the kingdom.'

# Summarize and translate simultaneously
print('Summarize in Spanish:')
print(generate(story, lang_tag='summarize in Spanish:'))

print('\nTranslate to French and make it formal:')
print(generate(story, lang_tag='translate to formal French:'))

print('\nConvert to Chinese and make it poetic:')
print(generate(story, lang_tag='translate to poetic Chinese:'))